In [ ]:
USE_COLAB = True

In [ ]:
if USE_COLAB:
    # let's install some libraries
    !pip install catboost plotly

In [ ]:
if USE_COLAB:
    # workaround for making plotly be able to render pointclouds in colab
    import IPython
    def configure_plotly_browser_state():
      display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
                },
              });
            </script>
            '''))

    IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

#### Data downloading ####

(Alternative link for manual downloading only [https://yadi.sk/d/CBoVCVIxJ2q2cw])

In [ ]:
def extract_zip_archive(path_to_archive, dst_folder='.'):
    import zipfile
    with zipfile.ZipFile(path_to_archive, 'r') as z:
      z.extractall(dst_folder)

import os
if not os.path.exists('snow.zip'):
    !wget https://www.dropbox.com/s/fymdgsc3vwvz957/snow.zip?dl=0 -O snow.zip
    extract_zip_archive('snow.zip')

In [ ]:
if USE_COLAB:
  # see tutorial at  https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2
  from google.colab import drive
  drive.mount('/content/gdrive')

# Lidar data segmentation

### What does lidar look like

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('Pa-q5elS_nE')

## What do we obtain from lidar

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import tqdm
py.init_notebook_mode(connected=True)

EQUAL_ASPECT_RATIO_LAYOUT = dict(
    margin={
        'l': 0,
        'r': 0,
        'b': 0,
        't': 0
    }, scene=dict(
    aspectmode='data'
))


def color(x, cmap='Reds'):
    cmap = plt.get_cmap(cmap)
    x = (x - np.min(x)) / np.max(x)
    
    return cmap(x)

%matplotlib inline

In [ ]:
ds = pd.read_csv('snow.csv')
ds = ds.set_index(['scene_id'])
ds.head()

`snow.csv` contains labeled points from multiple scenes, so `scene_id` defines an identification of scene from which this point came. Let's check what are `intensity` and `ring`.

### Ring

![](https://eckop.com/wp-content/uploads/2018/07/LIDAR_02-1024x366.png)

Let's color points according to their ring

In [ ]:
scene = ds.loc[0]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.ring, 'tab20'),
    },
    'text': scene.ring
})

py.iplot(fig)


### Intensity

Intensity represents a strength of reflected signal.

![](https://res.mdpi.com/sensors/sensors-15-28099/article_deploy/html/images/sensors-15-28099-g001-1024.png)

In [ ]:
fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.intensity, 'seismic'),
    },
    'text': scene.intensity
})

py.iplot(fig)

# Snow filtering

### Labels

In [ ]:
scene = ds.loc[1]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.label, 'seismic'),
    },
    'text': scene.label
})

py.iplot(fig)

## Baseline 1 - heuristic

In [ ]:
def get_high_intensity_points_mask(intensity, limit=3):
    # TODO implement it

filtered_scene = scene[get_high_intensity_points_mask(scene.intensity)]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': filtered_scene.x,
    'y': filtered_scene.y,
    'z': filtered_scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(filtered_scene.intensity, 'seismic'),
    },
    'text': filtered_scene.intensity
})

py.iplot(fig)

Not bad, but some snow points are still be in pointcloud while some obstacle points were filtered.

Can we improve the baseline? Yes! Let's train catboost (or lightgbm / GBDT / Random Forest / Neural network / whatsoever) to segment pointcloud.

## Baseline 2 - catboost on hand-crafted features

### Features

The previous baseline model processed each point independently and used just point intesity as a feature.
Let's use more features and process each point with its neigbourhood.

In [ ]:
from sklearn.neighbors import KDTree

class FeaturesExtractor(object):
    def __init__(self, r=1.0):
        self.xyz = None
        self.intensity = None
        self.ring = None
        self.index = None
        self.r = r

    def _feature_names(self):
        """
        Returns a list of feature names
        """
        names = [
            # TODO set of strings with names go here
        ]
        return ['{}_{}'.format(name, self.r) for name in names]

    def compute_point_features(self, point_id, neighbours):
        """
        Returns a list of features for given neighbours
        'neighbours' - list of indices
        """
        # TODO implement it
        
        # NOTE if you want to work with point coordinates here, 
        # it is a good idea to normalize these coordinates
        # to have zero at point of interest:
        #neighbour_points_xyz = self.xyz[neighbours]
        #point_xyz = self.xyz[point_id]
        #neighbour_points_xyz = neighbour_points_xyz - point_xyz
        
        features = [
            # TODO
        ]
        return features

    def get_point_neighbours(self, point_id):
        """
        Returns a list of indices of neighbour points
        """
        return self.index.query_radius(self.xyz[point_id][np.newaxis, :], r=self.r)[0]

    def __call__(self, xyz, intensity, ring):
        self.xyz = xyz[:]
        self.intensity = intensity[:]
        self.ring = ring[:]
        
        self.index = KDTree(self.xyz)
                
        features = []
        for point_id in range(len(self.xyz)):
            neighbours = self.get_point_neighbours(point_id)
            features.append(self.compute_point_features(point_id, neighbours))
        return pd.DataFrame(columns=self._feature_names(), data=features)

In [ ]:
import os

# FIXME
FEATURES_FOLDER = 'features' if not USE_COLAB else "/content/gdrive/My Drive/y_data_sdc_hw2_features"
if not os.path.exists(FEATURES_FOLDER):
    os.mkdir(FEATURES_FOLDER)

In [ ]:
features_extractor = FeaturesExtractor(r=1.0)

for scene_id in tqdm.tqdm(ds.reset_index().scene_id.unique()):
    scene = ds.loc[scene_id]
    features_df = \
        features_extractor(scene[['x', 'y', 'z']].values, scene.intensity.values, scene.ring.values)
    features_df['label'] = scene.label.values
    features_df.to_csv(os.path.join(FEATURES_FOLDER, '{}.csv'.format(scene_id)), index=False)

In [ ]:
ds_features = []
for scene in os.listdir(FEATURES_FOLDER):
    scene_features = pd.read_csv(os.path.join(FEATURES_FOLDER, scene))
    scene_id = int(os.path.splitext(scene)[0])
    scene_features['scene_id'] = scene_id
    ds_features.append(scene_features)
    
ds_features = pd.concat(ds_features)

In [ ]:
ds_features.to_csv('./snow_features.csv', index=False)

In [ ]:
ds_features.head()

In [ ]:
# ds_features = pd.read_csv('./snow_features.csv')

### Catboost training

In [ ]:
ds_features.shape

#### Train/val/test split
How to split - by scenes or by points?

In [ ]:
train = None # TODO train test split
test = None
val = None

#### Training

In [ ]:
import catboost

def learn(X_train, X_val, y_train, y_val):
    clf = catboost.CatBoostClassifier(n_estimators=100)
    clf.fit(
        X_train, y_train, early_stopping_rounds=10,
        use_best_model=True, eval_set=(X_val.values, y_val.values), plot=True, verbose=False)
    return clf

X_train = train.drop(["scene_id", "label"], axis=1)
y_train = train.label


X_val = val.drop(["scene_id", "label"], axis=1)
y_val = val.label

In [ ]:
cls = learn(X_train, X_val, y_train, y_val)

#### Testing

In [ ]:
X_test = test.drop(['scene_id', 'label'], axis=1)
y_test = test.label

from sklearn.metrics import precision_recall_curve, precision_score, recall_score

def test_one(clf, X_test, y_test):
    y_test_hat = clf.predict_proba(X_test)
    pr, rec, thr = precision_recall_curve(y_test, y_test_hat[:, 1])
    ix = np.linspace(1, len(pr)-1, num=2000).astype(int)
    return pr[ix], rec[ix], thr[ix - 1]


def heuristic_filter_scoring():
    pr = []
    rec = []
    filter_range = list(range(1, 10))
    for i in filter_range:
        y_test_heuristic_hat = np.ones(len(X_test))
        y_test_heuristic_hat[get_high_intensity_points_mask(test.intensity, i)] = 0
        pr.append(precision_score(y_test, y_test_heuristic_hat))
        rec.append(recall_score(y_test, y_test_heuristic_hat))
        
    return pr, rec, filter_range

pr_bl, rec_bl, thr_bl = heuristic_filter_scoring()

def plot_pr_rec(*models):
    traces = []
    for model, clf, X_test, y_test in models:
        pr, rec, thr = test_one(clf, X_test, y_test)
        pr_rec = go.Scattergl(x = rec, y = pr, mode='lines', text=thr, name=model)
        traces.append(pr_rec)

    pr_rec_bl = go.Scatter(x = rec_bl, y = pr_bl, mode='lines+markers', text=thr_bl, name='Intensity BL')

    layout = go.Layout(
        title='Precission-recall',
        xaxis=dict(
            title='Recall'
        ),
        yaxis=dict(
            title='Precission'
        ))
    fig = go.Figure(
        data=traces + [pr_rec_bl],
        layout=layout)
    py.iplot(fig)
    
models = [('Catboost classifier', cls, X_test, y_test)]
plot_pr_rec(*models)

### Detector results visualization

In [ ]:
y_test_hat = cls.predict_proba(test.drop(['scene_id', 'label'], axis=1))[:, 1]  # confidence for class 1

In [ ]:
scene_id = # TODO scene_id from test_set should be here
scene = ds.loc[scene_id]
scene_predictions = y_test_hat[test.scene_id == scene_id]

In [ ]:
fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene_predictions, 'seismic'),
    },
    'text': scene_predictions
})

py.iplot(fig)

# Homework

Your task is to train a better model that detects snow points and **beats the quality of baseline 2**. Target metric: precision-at-recall 0.95 (see implementation below).

What you **can** do:
* Train pointnet-like neural network to classify point
* Play with features extraction. For example, we didn't use neighbour point coordinates, just intensities and rings.

What you **can't** do:
* Just tune catboost (or any other off-the-shelf machine learning model) parameters while keeping the same features as in baseline 2. Please, do some lidar-related experiments.

As a solution, please send 
1) ipython notebook (or python script) with training code and 
2) csv file with results on hold-out dataset: https://www.dropbox.com/s/1o0qk691njr5l6h/snow_test_X.csv .
Output csv file should contain two columns: `point_id` (from input file) and `confidence` with confidence from your detector.

In [8]:
def calculate_precision_at_recall(y_pred, y_true, minimal_recall=0.95):
    """
    Args:
      y_pred - np.array of shape (N,) with prediction confidences
      y_true - ground truth labels (0 or 1), np.array of shape (N,)
    """
    positive_points_mask = y_true == 1
    positive_points_conf = y_pred[positive_points_mask]
    n_positive_points = len(positive_points_conf)
    n_positive_points_to_skip = int((1. - minimal_recall) * n_positive_points)
    threshold = np.partition(positive_points_conf, kth=n_positive_points_to_skip)[n_positive_points_to_skip]
    accepted_points_mask = y_pred >= threshold
    return np.mean(y_true[accepted_points_mask].astype(np.float32))

In [ ]:
print (calculate_precision_at_recall(y_test_hat, test.label.values))

In [9]:
def save_results_to_file(output_file, point_ids, y_pred):
    assert len(point_ids) == len(y_pred)
    df = pd.DataFrame(columns=["point_id", 'confidence'], data=list(zip(point_ids, y_pred)))
    df.to_csv(output_file, index=False)